In [1]:
import pygadgetreader
import matplotlib.pyplot as plt
import numpy as np

import SimPy
import importlib
importlib.reload(SimPy)

from galpy.potential import vcirc
from galpy.util.bovy_conversion import mass_in_1010msol,mass_in_msol
from galpy.util import bovy_plot
bovy_plot.bovy_print(axes_labelsize=17.,text_fontsize=16.,xtick_labelsize=14.,ytick_labelsize=14.)

import tqdm
import sys

NLU= 1. #kpc
NTU= 9.77792 #Myr
NVU= 100.#km/s
NMU= 2.32503e9 #Msun

# Combine snapshot files in Partree format

Read in the halo, disc and then bulge particles each of the ten 1e8 particle simulations from the gadget snapshots and then write to file in the format: m x y z vx vy vz

In [10]:
for i in range(10):
    #GalIC position units are 1 kpc
    hx= pygadgetreader.readsnap('/epsen_data/scr/bennett/MyMW_1e9/snap_010-'+str(int((i+1)*1000)),'pos',ptype='dm')
    hv= pygadgetreader.readsnap('/epsen_data/scr/bennett/MyMW_1e9/snap_010-'+str(int((i+1)*1000)),'vel',ptype='dm')
    hm= pygadgetreader.readsnap('/epsen_data/scr/bennett/MyMW_1e9/snap_010-'+str(int((i+1)*1000)),'mass',ptype='dm')*1e10/10.
    if i==0:
        htofile= np.vstack([hm/NMU,hx.T/NLU,hv.T/NVU]).T.astype('float32')
        with open('/epsen_data/scr/bennett/MyMW_1e9/MyMW.00000','wb') as f:
            htofile.tofile(f,format='f4')
    else:
        htofile= np.vstack([hm/NMU,hx.T/NLU,hv.T/NVU]).T.astype('float32')
        with open('/epsen_data/scr/bennett/MyMW_1e9/MyMW.00000','ab') as f:
            htofile.tofile(f,format='f4')

for i in range(10):
    #GalIC velocity units are 1 km/s
    dx= pygadgetreader.readsnap('/epsen_data/scr/bennett/MyMW_1e9/snap_010-'+str(int((i+1)*1000)),'pos',ptype='disk')
    dv= pygadgetreader.readsnap('/epsen_data/scr/bennett/MyMW_1e9/snap_010-'+str(int((i+1)*1000)),'vel',ptype='disk')
    dm= pygadgetreader.readsnap('/epsen_data/scr/bennett/MyMW_1e9/snap_010-'+str(int((i+1)*1000)),'mass',ptype='disk')*1e10/10
    
    dtofile= np.vstack([dm/NMU,dx.T/NLU,dv.T/NVU]).T.astype('float32')
    with open('/epsen_data/scr/bennett/MyMW_1e9/MyMW.00000','ab') as f:
        dtofile.tofile(f,format='f4')
    
for i in range(10):
    #GalIC mass units are 10^10 Msun
    bx= pygadgetreader.readsnap('/epsen_data/scr/bennett/MyMW_1e9/snap_010-'+str(int((i+1)*1000)),'pos',ptype='bulge')
    bv= pygadgetreader.readsnap('/epsen_data/scr/bennett/MyMW_1e9/snap_010-'+str(int((i+1)*1000)),'vel',ptype='bulge')
    bm= pygadgetreader.readsnap('/epsen_data/scr/bennett/MyMW_1e9/snap_010-'+str(int((i+1)*1000)),'mass',ptype='bulge')*1e10/10

    btofile= np.vstack([bm/NMU,bx.T/NLU,bv.T/NVU]).T.astype('float32')
    with open('/epsen_data/scr/bennett/MyMW_1e9/MyMW.00000','ab') as f:
        btofile.tofile(f,format='f4')
        
del hx,hv,hm
del dx,dv,dm
del bx,bv,bm

del htofile,dtofile,btofile

Returning DM    Positions
Returning DM    Velocities
Returning DM    Mass
Returning DM    Positions
Returning DM    Velocities
Returning DM    Mass
Returning DM    Positions
Returning DM    Velocities
Returning DM    Mass
Returning DM    Positions
Returning DM    Velocities
Returning DM    Mass
Returning DM    Positions
Returning DM    Velocities
Returning DM    Mass
Returning DM    Positions
Returning DM    Velocities
Returning DM    Mass
Returning DM    Positions
Returning DM    Velocities
Returning DM    Mass
Returning DM    Positions
Returning DM    Velocities
Returning DM    Mass
Returning DM    Positions
Returning DM    Velocities
Returning DM    Mass
Returning DM    Positions
Returning DM    Velocities
Returning DM    Mass
Returning DISK  Positions
Returning DISK  Velocities
Returning DISK  Mass
Returning DISK  Positions
Returning DISK  Velocities
Returning DISK  Mass
Returning DISK  Positions
Returning DISK  Velocities
Returning DISK  Mass
Returning DISK  Positions
Returning DI

### Define a function that only reads in the disc particles

In [11]:
# Offset is in bytes and count is number of items
def read_disc():
    with open('/epsen_data/scr/bennett/MyMW_1e9/MyMW.00000','rb') as f:
        mxv= np.fromfile(f,dtype='f4',offset=int(4e8)*7*4,count=7*int(5e8))
        mxv= np.reshape(mxv,(int(5e8),7))
    return mxv

## Adjust for the centre of mass of all particles and the angular momentum of the disc

$$COM = \frac{\Sigma_i \left(m_i\cdot v_i\right)}{\Sigma_j m_j}$$

$$L_z= \vec{x}\times \left(m\cdot \vec{v}\right)$$

In [12]:
nump= int(1e9)
nslice= 10
ndisk= int(5e8)
mtot= 0
COMraw= 0
Ltot= np.array([0., 0., 0.]) 

for i in tqdm.trange(nslice):
    with open('/epsen_data/scr/bennett/MyMW_1e9/MyMW.00000','rb') as f:
        mxv= np.fromfile(f,dtype='f4',offset=i*7*int(nump/nslice)*4,count=7*int(nump/nslice))
        mxv= np.reshape(mxv,(int(nump/nslice),7))
        mtot+=np.sum(mxv[:,0])
        COMraw+=np.sum(mxv[:,0][:,None]*mxv[:,1:4],axis=0)
        
COM= COMraw/mtot

for i in tqdm.trange(nslice):
    with open('/epsen_data/scr/bennett/MyMW_1e9/MyMW.00000','rb') as f:
        mxv= np.fromfile(f,dtype='f4',offset=(int(4e8)*7+i*7*int(ndisk/nslice))*4,count=7*int(ndisk/nslice))
        mxv= np.reshape(mxv,(int(ndisk/nslice),7))
        Ltot+= np.sum(np.cross((mxv[:,1:4]-COM),mxv[:,0][:,None]*mxv[:,4:]),axis=0)
        


100%|██████████| 10/10 [04:35<00:00, 27.57s/it]

100%|██████████| 10/10 [00:51<00:00,  5.13s/it]


In [13]:
print(Ltot/np.sqrt(np.sum(Ltot**2)))

[-1.94644429e-07  1.44684092e-06  1.00000000e+00]


### Use the anglar momentum to rotate the positions and velocities of the particles

In [14]:
Ltot= Ltot/np.sqrt(np.sum(Ltot**2))
rot= SimPy.calc_Rot_matrix(Ltot)

for i in tqdm.trange(nslice):
    with open('/epsen_data/scr/bennett/MyMW_1e9/MyMW.00000','rb') as f:
        mxv= np.fromfile(f,dtype='f4',offset=i*7*int(nump/nslice)*4,count=7*int(nump/nslice))
        mxv= np.reshape(mxv,(int(nump/nslice),7))
        pos= np.matmul(rot,mxv[:,1:4].T-COM[:,None])
        vel= np.matmul(rot,mxv[:,4:].T)
        
        tofile= np.vstack([mxv[:,0],pos,vel]).T.astype('float32')
        if i==0:
            with open('/epsen_data/scr/bennett/MyMW_1e9/MyMW_adj.00000','wb') as f:
                tofile.tofile(f,format='f4')
        else:
            with open('/epsen_data/scr/bennett/MyMW_1e9/MyMW_adj.00000','ab') as f:
                tofile.tofile(f,format='f4')


100%|██████████| 10/10 [10:44<00:00, 64.44s/it]


In [15]:
# Truncate so no particles are outside of 200 kpc
with open('/epsen_data/scr/bennett/MyMW_1e9/MyMW_adj.00000','rb') as f:
    mxv= np.fromfile(f,dtype='f4')
    mxv= np.reshape(mxv,(int(len(mxv)/7),7))

### Calculate number of particles in each component after truncation

In [16]:
r= np.sqrt(mxv[:,1]**2+mxv[:,2]**2+mxv[:,2]**2)
print('Halo particles: ',np.sum([r[:int(4e8)]<200]))
print('Disc particles: ',np.sum([r[int(4e8):int(9e8)]<200]))
print('Bulge particles: ',np.sum([r[int(9e8):]<200]))
print('Total number of particles: ',np.sum([r<200]))

mxv=mxv[r<200]

Halo particles:  313637500
Disc particles:  500000000
Bulge particles:  99492615
Total number of particles:  913130115


### Write truncated particles to file

In [17]:
with open('/epsen_data/scr/bennett/MyMW_1e9/MyMW_trunc.00000','wb') as f:
    mxv.tofile(f,format='f4')